# SOLUTION USING GENETIC ALGORITHM


## Version 1: Multiple chargers without considering uncertainty

Vehicle class

In [ ]:
# Class representing an electric vehicle
class ElectricVehicle:
    def __init__(self, id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate):
        self.id = id  # Vehicle identifier
        self.distance = distance  # Distance the vehicle needs to travel
        self.available_time_start = available_time_start  # Start time available for charging
        self.available_time_end = available_time_end  # End time available for charging
        self.current_charge = current_charge  # Current charge level
        self.battery_capacity = battery_capacity  # Battery capacity
        self.charge_speed = charge_speed  # Charging speed
        self.discharge_rate = discharge_rate  # KW discharged per 100 km (efficiency)

    # Calculates and returns the time needed to charge the vehicle enough for the specified distance
    def get_charging_time(self):
        required_charge = self.distance * (self.discharge_rate/100)  # Calculation of the required charge for the distance to be covered

        #Si el coche no tiene carga suficiente
        if self.current_charge < required_charge:
          required_charge -= self.current_charge  # Subtract the current charge
          return required_charge / self.charge_speed  # Time in hours required to charge the vehicle

        else:
          return 0

    # Calculates the time required to charge the vehicle
    def get_charging_cost(self, begin_time, end_time, hourly_prices):
        total_cost = 0
        current_time = begin_time
        while current_time < end_time:
            total_cost += self.charge_speed * hourly_prices[int(current_time)]
            current_time += 1
        return total_cost

    def __str__(self):
      return f"Vehículo {self.id}"



Class representing a element of solution list

In [ ]:
# Class representing a element of solution list
class item_planlist:
    def __init__(self, vehiculo, begin_time, end_time, cargador ):
        self.vehiculo = vehiculo
        self.tiempo_inicio = begin_time
        self.tiempo_fin = end_time
        self.cargador = cargador


Definition of the charger class. It consists of a vector containing the occupation periods

In [ ]:
# Class representing a charger
class Cargador:
    def __init__(self, id):
        self.id = id
        self.periodos_ocupado = []

    def set_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado.append((hora_inicio, hora_fin))

    def eliminar_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado = [(inicio, fin) for inicio, fin in self.periodos_ocupado if (inicio, fin) != (hora_inicio, hora_fin)]

    def liberar_cargador(self):
        self.periodos_ocupado = []

    def esta_ocupado(self, hora_inicio, hora_fin):
        for inicio, fin in self.periodos_ocupado:
            if inicio <= hora_fin and fin >= hora_inicio:
                return True
        return False

    def tiene_horas_ocupadas(self):
        return bool(self.periodos_ocupado)



In [ ]:
import random
import numpy as np

Funciones necesarias para el funcionamiento del algoritmo.

In [ ]:
#Calculates timespan
def calcular_timespan(solucion):
    tiempo_inicio_mas_temprano = min(item.tiempo_inicio for item in solucion)
    tiempo_fin_mas_tardio = max(item.tiempo_fin for item in solucion)
    return tiempo_fin_mas_tardio - tiempo_inicio_mas_temprano

Fundamental methods of the genetic algorithm

In [ ]:
#Generates the initial population
def generar_poblacion_inicial(vehiculos, cargadores, population_size):
    #Conjunto de soluciones
    poblacion = []
    for _ in range(population_size):

        #Generar cada solución
        solucion = []

        #Release chargers before generating a new solution.
        for i in range(len(cargadores)):
          cargadores[i].liberar_cargador()


        for vehiculo in vehiculos:
          #Check if the vehicle needs to be loaded
          if vehiculo.get_charging_time() > 0:

            tiempo_inicio = round(random.uniform(vehiculo.available_time_start, vehiculo.available_time_end-0.5),2)
            tiempo_fin = round(random.uniform(tiempo_inicio+0.5,tiempo_inicio+vehiculo.get_charging_time()),2)
            tiempo_fin = min(tiempo_fin, vehiculo.available_time_end)   #Ensure that the loading time does not exceed the available time.

            cargador = None

            #Assign charge
            for i in range(len(cargadores)):
              if(cargadores[i].esta_ocupado(tiempo_inicio, tiempo_fin) == False):
                cargador = cargadores[i]
                cargadores[i].set_estado(tiempo_inicio, tiempo_fin)
                break

            if(cargador != None):
              it = item_planlist(vehiculo,tiempo_inicio,tiempo_fin,cargador)
              solucion.append(it)

         #Add solution to popolation
        poblacion.append(solucion)
    return poblacion

#Evaluate the population in terms of cost and timespan
def evaluar_poblacion(poblacion, hourly_prices):
    #De un cargador a otro no varía el coste ni el tiempo que tarda
    evaluaciones = []
    for solucion in poblacion:
        coste_total = 0
        tiempo_total = 0
        for item in solucion:
            coste_total += item.vehiculo.get_charging_cost(item.tiempo_inicio, item.tiempo_fin, hourly_prices)
            tiempo_total += (item.tiempo_fin - item.tiempo_inicio)

        timespan = calcular_timespan(solucion)

        evaluaciones.append((solucion, coste_total, tiempo_total, timespan))
    return evaluaciones


#Select the best solutions for subsequent crossover
def seleccionar_padres(evaluaciones, num_padres):
    evaluaciones.sort(key=lambda x: (x[1], x[3]))
    padres = []
    for i in range(num_padres):
        padres.append(evaluaciones[i][0])

    return padres


#Crossover of parents
def cruzar_padres(padre1, padre2):
    hijo = []

    #Release chargers before generating a new solution.
    for i in range(len(cargadores)):
        cargadores[i].liberar_cargador()

    for i in range(len(padre1)):

        tiempo_inicio_hijo = min(padre1[i].tiempo_inicio, padre2[i].tiempo_inicio)
        tiempo_fin_hijo = max(padre1[i].tiempo_fin, padre2[i].tiempo_fin)

        cargador = None

        #Assign charger
        for j in range(len(cargadores)):
            if(cargadores[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False):
              cargador = cargadores[j]
              cargadores[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
              break

        item = item_planlist(padre1[i].vehiculo, tiempo_inicio_hijo, tiempo_fin_hijo, cargador)

        if (cargador == None):
              item = item_planlist(padre1[i].vehiculo, padre1[i].tiempo_inicio, padre1[i].tiempo_fin, padre1[i].cargador)

        hijo.append(item)

    return hijo



def mutar_hijo(hijo, mutation_rate):

    #Liberar los cargadores antes de generar una nueva solución
    for i in range(len(cargadores)):
      cargadores[i].liberar_cargador()

    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            nuevo_tiempo_inicio = round(random.uniform(hijo[i].vehiculo.available_time_start, hijo[i].vehiculo.available_time_end - 0.5),2)
            nuevo_tiempo_fin = round(nuevo_tiempo_inicio + hijo[i].vehiculo.get_charging_time(), 2)
            nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[i].vehiculo.available_time_end)

            if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5

            #Assign charger
            for j in range(len(cargadores)):
              if(cargadores[j].esta_ocupado(nuevo_tiempo_inicio, nuevo_tiempo_fin) == False):
                cargadores[j].set_estado(nuevo_tiempo_inicio, nuevo_tiempo_fin)
                hijo[i].tiempo_inicio = nuevo_tiempo_inicio
                hijo[i].tiempo_fin = nuevo_tiempo_fin
                hijo[i].cargador = cargadores[j]
                break

    return hijo


Inicialización de los vehículos de ejemplo y ejecución del algoritmo.

In [ ]:
import time

# Parameters of the genetic algorithm
population_size = 10
num_generations = 50
num_padres = 3
mutation_rate = 0.2

# Generate chargers list
cargadores = [Cargador(1), Cargador(2), Cargador(3)]

# Example of vehicles and prices per hour - Arrival and departure time with mean and variance
# Example of vehicles
vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 13, 1, 120, 12, 6)
vehiculo5 = ElectricVehicle(5, 100, 14, 17, 2, 80, 10, 5)
vehiculo6 = ElectricVehicle(6, 200, 13, 18, 3, 100, 8, 4)
vehiculo7 = ElectricVehicle(7, 150, 12, 20, 1, 120, 12, 6)
vehiculo8 = ElectricVehicle(8, 200, 15, 22, 1, 120, 12, 6)
vehiculo9 = ElectricVehicle(9, 100, 9, 10, 2, 80, 10, 5)
vehiculo10 = ElectricVehicle(10, 200, 8, 20, 3, 100, 8, 4)
vehiculo11 = ElectricVehicle(11, 150, 12, 18, 1, 120, 12, 6)
vehiculo12 = ElectricVehicle(12, 200, 14, 20, 1, 120, 12, 6)
vehiculo13 = ElectricVehicle(13, 100, 21, 22, 2, 80, 10, 5)
vehiculo14 = ElectricVehicle(14, 200, 22, 23, 3, 100, 8, 4)
vehiculo15 = ElectricVehicle(15, 150, 20, 23, 1, 120, 12, 6)
vehiculo16 = ElectricVehicle(16, 200, 15, 22, 1, 120, 12, 6)
vehiculo17 = ElectricVehicle(17, 100, 9, 20, 2, 80, 10, 5)
vehiculo18 = ElectricVehicle(18, 200, 6, 10, 3, 100, 8, 4)
vehiculo19 = ElectricVehicle(19, 150, 4, 9, 1, 120, 12, 6)
vehiculo20 = ElectricVehicle(20, 200, 17, 23, 1, 120, 12, 6)





'''vehiculo41 = ElectricVehicle(41, 100, 9, 10, 2, 80, 10, 5)
vehiculo42 = ElectricVehicle(42, 200, 8, 20, 3, 100, 8, 4)
vehiculo43 = ElectricVehicle(43, 150, 12, 18, 1, 120, 12, 6)
vehiculo44 = ElectricVehicle(44, 200, 14, 20, 1, 120, 12, 6)
vehiculo45 = ElectricVehicle(45, 100, 21, 22, 2, 80, 10, 5)
vehiculo46 = ElectricVehicle(46, 200, 22, 23, 3, 100, 8, 4)
vehiculo47 = ElectricVehicle(47, 150, 20, 23, 1, 120, 12, 6)
vehiculo48 = ElectricVehicle(48, 200, 15, 22, 1, 120, 12, 6)
vehiculo49 = ElectricVehicle(49, 100, 9, 20, 2, 80, 10, 5)
vehiculo50 = ElectricVehicle(50, 200, 6, 10, 3, 100, 8, 4)
vehiculo51 = ElectricVehicle(51, 150, 4, 9, 1, 120, 12, 6)
vehiculo52 = ElectricVehicle(52, 200, 17, 23, 1, 120, 12, 6)
vehiculo53 = ElectricVehicle(53, 100, 14, 18, 2, 80, 10, 5)
vehiculo54 = ElectricVehicle(54, 200, 18, 23, 3, 100, 8, 4)
vehiculo55 = ElectricVehicle(55, 150, 12, 15, 1, 120, 12, 6)
vehiculo56 = ElectricVehicle(56, 200, 15, 22, 1, 120, 12, 6)
vehiculo57 = ElectricVehicle(57, 100, 2, 10, 2, 80, 10, 5)
vehiculo58 = ElectricVehicle(58, 200, 3, 12, 3, 100, 8, 4)
vehiculo59 = ElectricVehicle(59, 150, 6, 7, 1, 120, 12, 6)
vehiculo60 = ElectricVehicle(60, 200, 7, 18, 1, 120, 12, 6)
vehiculo61 = ElectricVehicle(61, 100, 21, 22, 2, 80, 10, 5)
vehiculo62 = ElectricVehicle(62, 200, 5, 7, 3, 100, 8, 4)
vehiculo63 = ElectricVehicle(63, 150, 6, 8, 1, 120, 12, 6)
vehiculo64 = ElectricVehicle(64, 200, 5, 18, 1, 120, 12, 6)
vehiculo65 = ElectricVehicle(65, 100, 9, 10, 2, 80, 10, 5)
vehiculo66 = ElectricVehicle(66, 200, 8, 9, 3, 100, 8, 4)
vehiculo67 = ElectricVehicle(67, 150, 12, 13, 1, 120, 12, 6)
vehiculo68 = ElectricVehicle(68, 200, 12, 13, 1, 120, 12, 6)
vehiculo69 = ElectricVehicle(69, 100, 14, 17, 2, 80, 10, 5)
vehiculo70 = ElectricVehicle(70, 200, 13, 18, 3, 100, 8, 4)
vehiculo71 = ElectricVehicle(71, 150, 12, 20, 1, 120, 12, 6)
vehiculo72 = ElectricVehicle(72, 200, 15, 22, 1, 120, 12, 6)
vehiculo73 = ElectricVehicle(73, 100, 9, 10, 2, 80, 10, 5)
vehiculo74 = ElectricVehicle(74, 200, 8, 20, 3, 100, 8, 4)
vehiculo75 = ElectricVehicle(75, 150, 12, 18, 1, 120, 12, 6)
vehiculo76 = ElectricVehicle(76, 200, 14, 20, 1, 120, 12, 6)
vehiculo77 = ElectricVehicle(77, 100, 21, 22, 2, 80, 10, 5)
vehiculo78 = ElectricVehicle(78, 200, 22, 23, 3, 100, 8, 4)
vehiculo79 = ElectricVehicle(79, 150, 20, 23, 1, 120, 12, 6)
vehiculo80 = ElectricVehicle(80, 200, 15, 22, 1, 120, 12, 6)
vehiculo81 = ElectricVehicle(81, 100, 9, 20, 2, 80, 10, 5)
vehiculo82 = ElectricVehicle(82, 200, 6, 10, 3, 100, 8, 4)
vehiculo83 = ElectricVehicle(83, 150, 4, 9, 1, 120, 12, 6)
vehiculo84 = ElectricVehicle(84, 200, 17, 23, 1, 120, 12, 6)
vehiculo85 = ElectricVehicle(85, 100, 14, 18, 2, 80, 10, 5)
vehiculo86 = ElectricVehicle(86, 200, 18, 23, 3, 100, 8, 4)
vehiculo87 = ElectricVehicle(87, 150, 12, 15, 1, 120, 12, 6)
vehiculo88 = ElectricVehicle(88, 200, 15, 22, 1, 120, 12, 6)
vehiculo89 = ElectricVehicle(89, 100, 2, 10, 2, 80, 10, 5)
vehiculo90 = ElectricVehicle(90, 200, 3, 12, 3, 100, 8, 4)
vehiculo91 = ElectricVehicle(91, 150, 6, 7, 1, 120, 12, 6)
vehiculo92 = ElectricVehicle(92, 200, 7, 18, 1, 120, 12, 6)
vehiculo93 = ElectricVehicle(93, 100, 21, 22, 2, 80, 10, 5)
vehiculo94 = ElectricVehicle(94, 200, 5, 7, 3, 100, 8, 4)
vehiculo95 = ElectricVehicle(95, 150, 6, 8, 1, 120, 12, 6)
vehiculo96 = ElectricVehicle(96, 200, 5, 18, 1, 120, 12, 6)
vehiculo97 = ElectricVehicle(97, 100, 9, 10, 2, 80, 10, 5)
vehiculo98 = ElectricVehicle(98, 200, 8, 9, 3, 100, 8, 4)
vehiculo99 = ElectricVehicle(99, 150, 12, 13, 1, 120, 12, 6)
vehiculo100 = ElectricVehicle(100, 200, 12, 13, 1, 120, 12, 6)
vehiculo101 = ElectricVehicle(101, 100, 14, 17, 2, 80, 10, 5)
vehiculo102 = ElectricVehicle(102, 200, 13, 18, 3, 100, 8, 4)
vehiculo103 = ElectricVehicle(103, 150, 12, 20, 1, 120, 12, 6)
vehiculo104 = ElectricVehicle(104, 200, 15, 22, 1, 120, 12, 6)
vehiculo105 = ElectricVehicle(105, 100, 9, 10, 2, 80, 10, 5)
vehiculo106 = ElectricVehicle(106, 200, 8, 20, 3, 100, 8, 4)
vehiculo107 = ElectricVehicle(107, 150, 12, 18, 1, 120, 12, 6)
vehiculo108 = ElectricVehicle(108, 200, 14, 20, 1, 120, 12, 6)
vehiculo109 = ElectricVehicle(109, 100, 21, 22, 2, 80, 10, 5)
vehiculo110 = ElectricVehicle(110, 200, 22, 23, 3, 100, 8, 4)
vehiculo111 = ElectricVehicle(111, 150, 20, 23, 1, 120, 12, 6)
vehiculo112 = ElectricVehicle(112, 200, 15, 22, 1, 120, 12, 6)
vehiculo113 = ElectricVehicle(113, 100, 9, 20, 2, 80, 10, 5)
vehiculo114 = ElectricVehicle(114, 200, 6, 10, 3, 100, 8, 4)
vehiculo115 = ElectricVehicle(115, 150, 4, 9, 1, 120, 12, 6)
vehiculo116 = ElectricVehicle(116, 200, 17, 23, 1, 120, 12, 6)
vehiculo117 = ElectricVehicle(117, 100, 14, 18, 2, 80, 10, 5)
vehiculo118 = ElectricVehicle(118, 200, 18, 23, 3, 100, 8, 4)
vehiculo119 = ElectricVehicle(119, 150, 12, 15, 1, 120, 12, 6)
vehiculo120 = ElectricVehicle(120, 200, 15, 22, 1, 120, 12, 6)
vehiculo121 = ElectricVehicle(121, 100, 9, 10, 2, 80, 10, 5)
vehiculo122 = ElectricVehicle(122, 200, 8, 9, 3, 100, 8, 4)
vehiculo123 = ElectricVehicle(123, 150, 12, 13, 1, 120, 12, 6)
vehiculo124 = ElectricVehicle(124, 200, 12, 13, 1, 120, 12, 6)
vehiculo125 = ElectricVehicle(125, 100, 14, 17, 2, 80, 10, 5)
vehiculo126 = ElectricVehicle(126, 200, 13, 18, 3, 100, 8, 4)
vehiculo127 = ElectricVehicle(127, 150, 12, 20, 1, 120, 12, 6)
vehiculo128 = ElectricVehicle(128, 200, 15, 22, 1, 120, 12, 6)
vehiculo129 = ElectricVehicle(129, 100, 9, 10, 2, 80, 10, 5)
vehiculo130 = ElectricVehicle(130, 200, 8, 20, 3, 100, 8, 4)
vehiculo131 = ElectricVehicle(131, 150, 12, 18, 1, 120, 12, 6)
vehiculo132 = ElectricVehicle(132, 200, 14, 20, 1, 120, 12, 6)
vehiculo133 = ElectricVehicle(133, 100, 21, 22, 2, 80, 10, 5)
vehiculo134 = ElectricVehicle(134, 200, 22, 23, 3, 100, 8, 4)
vehiculo135 = ElectricVehicle(135, 150, 20, 23, 1, 120, 12, 6)
vehiculo136 = ElectricVehicle(136, 200, 15, 22, 1, 120, 12, 6)
vehiculo137 = ElectricVehicle(137, 100, 9, 20, 2, 80, 10, 5)
vehiculo138 = ElectricVehicle(138, 200, 6, 10, 3, 100, 8, 4)
vehiculo139 = ElectricVehicle(139, 150, 4, 9, 1, 120, 12, 6)
vehiculo140 = ElectricVehicle(140, 200, 17, 23, 1, 120, 12, 6)
vehiculo141 = ElectricVehicle(141, 100, 14, 18, 2, 80, 10, 5)
vehiculo142 = ElectricVehicle(142, 200, 18, 23, 3, 100, 8, 4)
vehiculo143 = ElectricVehicle(143, 150, 12, 15, 1, 120, 12, 6)
vehiculo144 = ElectricVehicle(144, 200, 15, 22, 1, 120, 12, 6)
vehiculo145 = ElectricVehicle(145, 100, 2, 10, 2, 80, 10, 5)
vehiculo146 = ElectricVehicle(146, 200, 3, 12, 3, 100, 8, 4)
vehiculo147 = ElectricVehicle(147, 150, 6, 7, 1, 120, 12, 6)
vehiculo148 = ElectricVehicle(148, 200, 7, 18, 1, 120, 12, 6)
vehiculo149 = ElectricVehicle(149, 100, 21, 22, 2, 80, 10, 5)
vehiculo150 = ElectricVehicle(150, 200, 5, 7, 3, 100, 8, 4)'''





'''vehiculo5 = ElectricVehicle(5, 100, 14, 17, 2, 80, 10, 5)
vehiculo6 = ElectricVehicle(6, 200, 13, 18, 3, 100, 8, 4)
vehiculo7 = ElectricVehicle(7, 150, 12, 20, 1, 120, 12, 6)
vehiculo8 = ElectricVehicle(8, 200, 15, 22, 1, 120, 12, 6)
vehiculo9 = ElectricVehicle(9, 100, 9, 10, 2, 80, 10, 5)
vehiculo10 = ElectricVehicle(10, 200, 8, 20, 3, 100, 8, 4)
vehiculo11 = ElectricVehicle(11, 150, 12, 18, 1, 120, 12, 6)
vehiculo12 = ElectricVehicle(12, 200, 14, 20, 1, 120, 12, 6)
vehiculo13 = ElectricVehicle(13, 100, 21, 22, 2, 80, 10, 5)
vehiculo14 = ElectricVehicle(14, 200, 22, 23, 3, 100, 8, 4)
vehiculo15 = ElectricVehicle(15, 150, 20, 23, 1, 120, 12, 6)
vehiculo16 = ElectricVehicle(16, 200, 15, 22, 1, 120, 12, 6)
vehiculo17 = ElectricVehicle(17, 100, 9, 20, 2, 80, 10, 5)
vehiculo18 = ElectricVehicle(18, 200, 6, 10, 3, 100, 8, 4)
vehiculo19 = ElectricVehicle(19, 150, 4, 9, 1, 120, 12, 6)
vehiculo20 = ElectricVehicle(20, 200, 17, 23, 1, 120, 12, 6)
vehiculo21 = ElectricVehicle(21, 100, 14, 18, 2, 80, 10, 5)
vehiculo22 = ElectricVehicle(22, 200, 18, 23, 3, 100, 8, 4)
vehiculo23 = ElectricVehicle(23, 150, 12, 15, 1, 120, 12, 6)
vehiculo24 = ElectricVehicle(24, 200, 15, 22, 1, 120, 12, 6)
vehiculo25 = ElectricVehicle(25, 100, 2, 10, 2, 80, 10, 5)
vehiculo26 = ElectricVehicle(26, 200, 3, 12, 3, 100, 8, 4)
vehiculo27 = ElectricVehicle(27, 150, 6, 7, 1, 120, 12, 6)
vehiculo28 = ElectricVehicle(28, 200, 7, 18, 1, 120, 12, 6)
vehiculo29 = ElectricVehicle(29, 100, 21, 22, 2, 80, 10, 5)
vehiculo30 = ElectricVehicle(30, 200, 5, 7, 3, 100, 8, 4)
vehiculo31 = ElectricVehicle(31, 150, 6, 8, 1, 120, 12, 6)
vehiculo32 = ElectricVehicle(32, 200, 5, 18, 1, 120, 12, 6)'''

vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4, vehiculo5, vehiculo6, vehiculo7, vehiculo8, vehiculo9, vehiculo10]


hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

inicio = time.time()

# Generar población inicial
poblacion = generar_poblacion_inicial(vehiculos, cargadores, population_size)

#Print population - Uncomment if necessary
'''print("POBLACION")
for sol in poblacion:
  for it in sol:
    print("identificador vehiculo: "+ str(it.vehiculo.id))
    print("inicio "+str(it.tiempo_inicio))
    print("fin "+str(it.tiempo_fin))
    print("cargador "+str(it.cargador.id))
    print("")

  print("-------------------------------------------------")'''

'''evaluaciones=evaluar_poblacion(poblacion,hourly_prices)
padres = seleccionar_padres(evaluaciones, num_padres)
padre1 = random.choice(padres)
padre2 = random.choice(padres)
hijo = cruzar_padres(padre1, padre2)
hijo = mutar_hijo(hijo, mutation_rate)'''



# Run genetic algorithm
for generation in range(num_generations):
    evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
    padres = seleccionar_padres(evaluaciones, num_padres)

    nueva_poblacion = []
    for i in range(population_size):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        hijo = cruzar_padres(padre1, padre2)
        hijo_mutado = mutar_hijo(hijo, mutation_rate)
        nueva_poblacion.append(hijo_mutado)

    poblacion = nueva_poblacion

# Get the best solution found
evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
evaluaciones.sort(key=lambda x: (x[1], x[3]))
mejor_sol = evaluaciones[0]

fin = time.time()

tiempo_transcurrido = fin - inicio

print("#########################################################################")

# Print the solutiuon
print("Coste total: " + str(mejor_sol[1]))
print("Tiempo total: " + str(mejor_sol[2]))
print("Timespan " + str(mejor_sol[3]) )
print("Planificación:")

for item in mejor_sol[0]:
  print("Identificador del vehículo: "+str(item.vehiculo.id))
  print("Hora inicio carga: "+str(item.tiempo_inicio))
  print("Hora fin carga: "+str(item.tiempo_fin))
  print("Cargador asignado: "+str(item.cargador.id))
  print("")


print(tiempo_transcurrido)














Visualization of results.

In [ ]:
import matplotlib.pyplot as plt

def plot_gantt_chart(solution, cargadores):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Establecer límites y etiquetas del eje y
    ax.set_ylim(0, len(cargadores))
    ax.set_yticks(range(len(cargadores)))
    ax.set_yticklabels([f"Cargador {cargador.id}" for cargador in cargadores], fontsize=12)

    # Establecer límites y etiquetas del eje x
    ax.set_xlim(0, 24)
    ax.set_xticks(range(0, 25, 2))
    ax.set_xlabel("Horas del día", fontsize=14)

    # Establecer separación de las barras del eje x
    bar_height = 0.4

    # Colorear las barras según los tiempos de inicio y fin de cada vehículo
    for i, vehiculo in enumerate(solution):
        inicio = vehiculo['tiempo_inicio']
        fin = vehiculo['tiempo_fin']
        cargador_id = vehiculo['cargador_id']
        cargador_index = [cargador.id for cargador in cargadores].index(cargador_id)

        ax.barh(cargador_index, fin - inicio, left=inicio, height=bar_height, align='center', alpha=0.8)
        ax.text(inicio + (fin - inicio) / 2, cargador_index + bar_height / 2, f"Vehículo {vehiculo['id']}", ha='center', va='center', fontsize=10, color='black')

    plt.grid(axis='x')
    plt.title("Diagrama de Gantt - Planificación de Carga", fontsize=16)

    plt.tight_layout()
    plt.show()

# Obtener la mejor solución encontrada
evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
evaluaciones.sort(key=lambda x: (x[1], x[2]))
mejor_sol = evaluaciones[0]

# Preparar datos para el diagrama de Gantt
datos_gantt = []
for item in mejor_sol[0]:
    vehiculo_info = {
        'id': item.vehiculo.id,
        'tiempo_inicio': item.tiempo_inicio,
        'tiempo_fin': item.tiempo_fin,
        'cargador_id': item.cargador.id,
    }
    datos_gantt.append(vehiculo_info)

# Llamar a la función para mostrar el diagrama de Gantt de la mejor solución encontrada
plot_gantt_chart(datos_gantt, cargadores)


##Version 2: Uncertainty in the arrival time of vehicles is incorporated

Normal probability distribution is used for the arrival time of vehicles (not incorporated in the objective function but calculated a priori)

In [ ]:
# Generar población inicial
def generar_poblacion_inicial_dist(vehiculos, cargadores, population_size):
    poblacion = []
    for _ in range(population_size):
        solucion = []

        # Liberar los cargadores antes de generar una nueva solución
        for cargador in cargadores:
            cargador.liberar_cargador()

        for vehiculo in vehiculos:
            if vehiculo.get_charging_time() > 0:

                #MODIFICACIÓN
                # Generar hora de llegada aleatoria usando distribución normal
                tiempo_llegada = np.random.normal(vehiculo.available_time_start, vehiculo.available_time_end - vehiculo.available_time_start)

                # Asegurar que el tiempo de llegada esté dentro de los límites permitidos
                tiempo_llegada = max(vehiculo.available_time_start, min(vehiculo.available_time_end, tiempo_llegada))

                tiempo_fin = tiempo_llegada + vehiculo.get_charging_time()
                tiempo_fin = min(tiempo_fin, vehiculo.available_time_end)

                cargador = None
                for c in cargadores:
                    if not c.esta_ocupado(tiempo_llegada, tiempo_fin):
                        cargador = c
                        c.set_estado(tiempo_llegada, tiempo_fin)
                        break

                if cargador is not None:
                    it = item_planlist(vehiculo, tiempo_llegada, tiempo_fin, cargador)
                    solucion.append(it)

        poblacion.append(solucion)
    return poblacion



def evaluar_poblacion(poblacion, hourly_prices):
    #De un cargador a otro no varía el coste ni el tiempo que tarda
    evaluaciones = []
    for solucion in poblacion:
        coste_total = 0
        tiempo_total = 0
        for item in solucion:
            coste_total += item.vehiculo.get_charging_cost(item.tiempo_inicio, item.tiempo_fin, hourly_prices)
            tiempo_total += (item.tiempo_fin - item.tiempo_inicio)

        timespan = calcular_timespan(solucion)

        evaluaciones.append((solucion, coste_total, tiempo_total, timespan))

    return evaluaciones



def seleccionar_padres(evaluaciones, num_padres):
    evaluaciones.sort(key=lambda x: (x[1], x[2]))
    padres = []
    for i in range(num_padres):
        padres.append(evaluaciones[i][0])
    return padres



def cruzar_padres(padre1, padre2):
    hijo = []

    #Liberar los cargadores antes de generar una nueva solución
    for i in range(len(cargadores)):
        cargadores[i].liberar_cargador()

    for i in range(len(padre1)):

        tiempo_inicio_hijo = min(padre1[i].tiempo_inicio, padre2[i].tiempo_inicio)
        tiempo_fin_hijo = max(padre1[i].tiempo_fin, padre2[i].tiempo_fin)

        cargador = None
        #Asignar cargador
        for j in range(len(cargadores)):
            if(cargadores[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False):
              cargador = cargadores[j]
              cargadores[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
              break

        item = item_planlist(padre1[i].vehiculo, tiempo_inicio_hijo, tiempo_fin_hijo, cargador)

        if (cargador == None):
              item = item_planlist(padre1[i].vehiculo, padre1[i].tiempo_inicio, padre1[i].tiempo_fin, padre1[i].cargador)

        hijo.append(item)

    return hijo



def mutar_hijo(hijo, mutation_rate):

    #Liberar los cargadores antes de generar una nueva solución
    for i in range(len(cargadores)):
      cargadores[i].liberar_cargador()

    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            nuevo_tiempo_inicio = random.uniform(hijo[i].vehiculo.available_time_start, hijo[i].vehiculo.available_time_end - 0.5)
            nuevo_tiempo_fin = nuevo_tiempo_inicio + hijo[i].vehiculo.get_charging_time()
            nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[i].vehiculo.available_time_end)

            if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5

            #Asignar cargador
            for j in range(len(cargadores)):
              if(cargadores[j].esta_ocupado(nuevo_tiempo_inicio, nuevo_tiempo_fin) == False):
                cargadores[j].set_estado(nuevo_tiempo_inicio, nuevo_tiempo_fin)
                hijo[i].tiempo_inicio = nuevo_tiempo_inicio
                hijo[i].tiempo_fin = nuevo_tiempo_fin
                hijo[i].cargador = cargadores[j]
                break

    return hijo


In [ ]:
# Parámetros del algoritmo genético
population_size = 10
num_generations = 50
num_padres = 10
mutation_rate = 0.2

# Generar la lista de cargadores
cargadores1 = [Cargador(1), Cargador(2), Cargador(3)]

# Ejemplo de vehículos con media y varianza para la hora de llegada
vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 14, 1, 120, 12, 6)

# Distribución normal para la hora de llegada (media, varianza)
#hora_llegada_media = [9, 8, 12, 12]
#hora_llegada_varianza = [0.25, 0.25, 0.25, 0.25]

lista_vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4]

hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

#Se crea esta función para poder ejecutarla con diferentes tamaños del problema sin tener que escribir todo el código de nuevo
def ejecutar_algoritmo_genetico(vehiculos, cargadores):
    # Generar población inicial
    poblacion = generar_poblacion_inicial_dist(vehiculos, cargadores, population_size)

    #Imprimir la poblción
    '''print("POBLACION")
    for sol in poblacion:
      for it in sol:
        print("identificador vehiculo: "+ str(it.vehiculo.id))
        print("inicio "+str(it.tiempo_inicio))
        print("fin "+str(it.tiempo_fin))
        print("cargador "+str(it.cargador.id))
        print("")

      print("-------------------------------------------------")'''

    evaluaciones=evaluar_poblacion(poblacion,hourly_prices)

    # Ejecutar algoritmo genético
    for generation in range(num_generations):
        evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
        padres = seleccionar_padres(evaluaciones, num_padres)

        nueva_poblacion = []
        for i in range(population_size):
            padre1 = random.choice(padres)
            padre2 = random.choice(padres)
            hijo = cruzar_padres(padre1, padre2)
            hijo_mutado = mutar_hijo(hijo, mutation_rate)
            nueva_poblacion.append(hijo_mutado)

        poblacion = nueva_poblacion

    # Obtener la mejor solución encontrada
    evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
    evaluaciones.sort(key=lambda x: (x[1], x[3]))
    mejor_sol = evaluaciones[0]

    print("#########################################################################")

    # Imprimir la mejor solución encontrada
    print("Coste total: "+str(mejor_sol[1]))
    print("Tiempo total: "+str(mejor_sol[2]))
    print("Timespan: "+str(mejor_sol[3]))
    print("Planificación:")

    for item in mejor_sol[0]:
      print("Identificador del vehículo: "+str(item.vehiculo.id))
      print("Hora inicio carga: "+str(item.tiempo_inicio))
      print("Hora fin carga: "+str(item.tiempo_fin))
      print("Cargador asignado: "+str(item.cargador.id))
      print("")

ejecutar_algoritmo_genetico(lista_vehiculos, cargadores1)

# Generar la lista de cargadores
cargadores2 = [Cargador(1), Cargador(2), Cargador(3), Cargador(4)]

# Ejemplo de vehículos con media y varianza para la hora de llegada
vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 5, 120, 12, 4)
vehiculo4 = ElectricVehicle(4, 200, 12, 14, 3, 100, 8, 7)
vehiculo5 = ElectricVehicle(5, 250, 12, 15, 2, 130, 10, 4)
vehiculo6 = ElectricVehicle(6, 120, 12, 16, 4, 150, 11, 5)
vehiculo7 = ElectricVehicle(7, 200, 17, 22, 6, 80, 12, 5)

# Distribución normal para la hora de llegada (media, varianza)
#hora_llegada_media = [9, 8, 12, 12]
#hora_llegada_varianza = [0.25, 0.25, 0.25, 0.25]

vehiculos2 = [vehiculo1, vehiculo2, vehiculo3, vehiculo4, vehiculo5, vehiculo6, vehiculo7]

inicio = time.time()
ejecutar_algoritmo_genetico(vehiculos2, cargadores2)
fin = time.time()

tiempo_transcurrido = fin - inicio
print(tiempo_transcurrido)








##Version 3: Alternative crossover and mutation strategies.

In [ ]:
import random

#Single point strategy
def cruzar_padres_single_point(padre1, padre2):
    hijo = []

    # Release chargers before generating a new solution.
    for cargador in cargadores:
        cargador.liberar_cargador()

    # Obtain a random crossover point
    punto_cruce = random.randint(0, len(padre1) - 1)

    # Find a crossover point that avoids overlaps
    while any(cargador.esta_ocupado(padre1[punto_cruce].tiempo_inicio, padre1[punto_cruce].tiempo_fin) for cargador in cargadores):
        punto_cruce = random.randint(0, len(padre1) - 1)

    # Apply single-point crossover
    for i in range(len(padre1)):
        if i < punto_cruce:
            tiempo_inicio_hijo = padre1[i].tiempo_inicio
            tiempo_fin_hijo = padre1[i].tiempo_fin
            cargador = padre1[i].cargador
        else:
            tiempo_inicio_hijo = padre2[i].tiempo_inicio
            tiempo_fin_hijo = padre2[i].tiempo_fin
            cargador = padre2[i].cargador

        # Asisign charger
        for j in range(len(cargadores)):
            if cargadores[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False:
                cargador = cargadores[j]
                cargadores[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
                break

        item = item_planlist(padre1[i].vehiculo, tiempo_inicio_hijo, tiempo_fin_hijo, cargador)
        hijo.append(item)

    return hijo


import random

# Inversion estrategy
def mutar_hijo_inversion(hijo, mutation_rate):
    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            # Random selection of indexes
            index1, index2 = random.sample(range(len(hijo)), 2)

            # Ensure that index1 < index2 for the subsequence inversion
            start_index = min(index1, index2)
            end_index = max(index1, index2)

            # Invert the gene subsequence
            subsecuencia_invertida = hijo[start_index:end_index+1][::-1]

            # Update the start and end times of loading in the solution
            for j in range(len(subsecuencia_invertida)):
                nuevo_tiempo_inicio = random.uniform(hijo[j + start_index].vehiculo.available_time_start, hijo[j + start_index].vehiculo.available_time_end - 0.5)
                nuevo_tiempo_fin = nuevo_tiempo_inicio + hijo[j + start_index].vehiculo.get_charging_time()
                nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[j + start_index].vehiculo.available_time_end)

                if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                    nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5

                hijo[j + start_index].tiempo_inicio = nuevo_tiempo_inicio
                hijo[j + start_index].tiempo_fin = nuevo_tiempo_fin

    return hijo


In [ ]:
# Parámetros del algoritmo genético
population_size = 10
num_generations = 50
num_padres = 10
mutation_rate = 0.2

# Generar la lista de cargadores
cargadores = [Cargador(1), Cargador(2), Cargador(3)]

# Ejemplo de vehículos con media y varianza para la hora de llegada
vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 14, 1, 120, 12, 6)

# Distribución normal para la hora de llegada (media, varianza)
#hora_llegada_media = [9, 8, 12, 12]
#hora_llegada_varianza = [0.25, 0.25, 0.25, 0.25]

vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4]

hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

# Generar población inicial
poblacion = generar_poblacion_inicial_dist(vehiculos, cargadores, population_size)

#Imprimir la poblción
print("POBLACION")
for sol in poblacion:
  for it in sol:
    print("identificador vehiculo: "+ str(it.vehiculo.id))
    print("inicio "+str(it.tiempo_inicio))
    print("fin "+str(it.tiempo_fin))
    print("cargador "+str(it.cargador.id))
    print("")

  print("-------------------------------------------------")

evaluaciones=evaluar_poblacion(poblacion,hourly_prices)

# Ejecutar algoritmo genético
for generation in range(num_generations):
    evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
    padres = seleccionar_padres(evaluaciones, num_padres)

    nueva_poblacion = []
    for i in range(population_size):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        hijo = cruzar_padres_single_point(padre1, padre2)
        hijo_mutado = mutar_hijo_inversion(hijo, mutation_rate)
        nueva_poblacion.append(hijo_mutado)

    poblacion = nueva_poblacion

# Obtener la mejor solución encontrada
evaluaciones = evaluar_poblacion(poblacion, hourly_prices)
evaluaciones.sort(key=lambda x: (x[1], x[2]))
mejor_sol = evaluaciones[0]

print("#########################################################################")

# Imprimir la mejor solución encontrada
print("Coste total: "+str(mejor_sol[1]))
print("Tiempo total: "+str(mejor_sol[2]))
print("Timespan: "+str(mejor_sol[3]))
print("Planificación:")

for item in mejor_sol[0]:
  print("Identificador del vehículo: "+str(item.vehiculo.id))
  print("Hora inicio carga: "+str(item.tiempo_inicio))
  print("Hora fin carga: "+str(item.tiempo_fin))
  print("Cargador asignado: "+str(item.cargador.id))
  print("")














# SOLUTION THROUGH LINEAR OPTIMIZATION WITH CONSTRAINTS (MILP)

Installation of the 'pulp' library required for the algorithm's operation

In [ ]:
%pip install pulp

## Version 0: Basic version of the optimization algorithm

Considers only one loader. Does not take into account current load or vehicle loading speed.

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value

# vehicle
class Vehicle:
    def __init__(self, distance, available_time_start, available_time_end, battery_capacity):
        self.distance = distance
        self.available_time_start = available_time_start
        self.available_time_end = available_time_end
        self.battery_capacity = battery_capacity

def optimize_charging_schedule(vehicles, hourly_prices):
    # Create the optimization problem
    problem = LpProblem("Charging_Schedule_Optimization", LpMinimize)

    # Decision variables
    charging_vars = []
    energy_vars = []

    for vehicle in vehicles:
        # Binary variable indicating whether the vehicle is loaded or not
        charge_var = LpVariable(f"Charge_{vehicle}", cat="Binary")
        charging_vars.append(charge_var)

        # Variables for the energy charged for each vehicle and hour
        energy_vars_vehicle = []
        for hour in range(vehicle.available_time_start, vehicle.available_time_end):
            energy_var = LpVariable(f"Energy_{vehicle}_{hour}", lowBound=0, upBound=vehicle.battery_capacity)
            energy_vars_vehicle.append(energy_var)

        energy_vars.append(energy_vars_vehicle)

    # Objective function: Minimize the total charging cost
    total_cost = lpSum([hourly_prices[hour] * energy_vars[i][hour - vehicles[i].available_time_start]
                        for i in range(len(vehicles))
                        for hour in range(vehicles[i].available_time_start, vehicles[i].available_time_end)])
    problem += total_cost

    # Constraints
    for i in range(len(vehicles)):
        for hour in range(vehicles[i].available_time_start, vehicles[i].available_time_end):
            # Constraint: The energy charged in each hour cannot exceed the vehicle's battery capacity
            problem += energy_vars[i][hour - vehicles[i].available_time_start] <= vehicles[i].battery_capacity * charging_vars[i]

    for hour in range(vehicles[0].available_time_start, vehicles[0].available_time_end):
        # Constraint: The sum of the charged energies in each hour must be equal to the distance the vehicle needs to travel
        problem += lpSum([energy_vars[i][hour - vehicles[i].available_time_start] for i in range(len(vehicles))]) == vehicles[0].distance

    # Solve the problem
    problem.solve()

    # Obtener los resultados
    charging_schedule = []
    for i in range(len(vehicles)):
        vehicle_schedule = []
        for hour in range(vehicles[i].available_time_start, vehicles[i].available_time_end):
            energy = value(energy_vars[i][hour - vehicles[i].available_time_start])
            vehicle_schedule.append((hour, energy))
        charging_schedule.append((vehicles[i], vehicle_schedule))

    total_cost = value(total_cost)

    return charging_schedule, total_cost

# Example
vehicles = [
    Vehicle(distance=100, available_time_start=0, available_time_end=8, battery_capacity=200),
    Vehicle(distance=80, available_time_start=2, available_time_end=10, battery_capacity=150),
    Vehicle(distance=120, available_time_start=6, available_time_end=14, battery_capacity=250)
]

hourly_prices = [10, 8, 12, 15, 12, 10, 8, 10, 12, 15, 10, 8, 10, 12, 15, 10, 8, 12, 15, 12, 10, 8, 10, 12]

charging_schedule, total_cost = optimize_charging_schedule(vehicles, hourly_prices)

print("Optimal Charging Schedule:")
for vehicle, schedule in charging_schedule:
    print(f"Vehicle: {vehicle.distance} km, Available Time: {vehicle.available_time_start}-{vehicle.available_time_end}")
    print("Hour\tEnergy")
    for hour, energy in schedule:
        print(f"{hour}\t{energy}")
    print("----------------------")

print("Total Cost:", total_cost)


## Version 1: Basic version taking into account charging speed and current load

Only one charger

In [ ]:
%pip install pulp


In [ ]:
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, value

class Vehicle:
    def __init__(self, distance, available_time_start, available_time_end, battery_capacity, current_charge, charging_speed, efficiency):
        self.distance = distance
        self.available_time_start = available_time_start
        self.available_time_end = available_time_end
        self.battery_capacity = battery_capacity
        self.current_charge = current_charge
        self.charging_speed = charging_speed
        self.efficiency = efficiency

def optimize_charging_schedule(vehicles, hourly_prices):
    problem = LpProblem("Charging_Schedule_Optimization", LpMinimize)

    charging_schedules = []
    total_cost = LpVariable("Total_Cost", lowBound=0)
    total_time = LpVariable("Total_Time", lowBound=0)

    for vehicle in vehicles:
        schedule = []
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            var_name = f"Vehicle_{vehicle.distance}_{hour}"
            var = LpVariable(var_name, lowBound=0, upBound=vehicle.battery_capacity)
            schedule.append(var)
        charging_schedules.append(schedule)

        # Restriction: Charged energy must be sufficient to cover the distance
        required_energy = vehicle.distance / (vehicle.efficiency/100)
        problem += lpSum(schedule) >= required_energy

        # Restriction: Charging time must be within available time
        charging_time = lpSum(schedule) / vehicle.charging_speed
        problem += charging_time <= vehicle.available_time_end - vehicle.available_time_start

    problem += total_cost + total_time



    # Constraint: total distance must be covered by the charging schedules
    for i, vehicle in enumerate(vehicles):
        problem += lpSum(charging_schedules[i]) * vehicle.charging_speed == vehicle.distance

    # Constraint: total time is the sum of hours * charging speed for each vehicle
    problem += total_time == lpSum(lpSum(charging_schedules[i]) / vehicle.charging_speed for i, vehicle in enumerate(vehicles))

    # Constraint: calculate total cost based on charging schedules and hourly prices
    for i, vehicle in enumerate(vehicles):
        problem += lpSum(charging_schedules[i][hour - vehicle.available_time_start] * hourly_prices[hour] for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)) == total_cost

    # Solve the problem
    problem.solve()

    # Extract the charging schedules and total cost
    charging_schedule = []
    for i, vehicle in enumerate(vehicles):
        schedule = [(hour, value(var)) for hour, var in zip(range(vehicle.available_time_start, vehicle.available_time_end + 1), charging_schedules[i])]
        charging_schedule.append((vehicle, schedule))

    total_cost = value(total_cost)

    return charging_schedule, total_cost


# Example usage
vehicles = [
    Vehicle(distance=100, available_time_start=0, available_time_end=6, battery_capacity=200, current_charge=50, charging_speed=10, efficiency=20),
    Vehicle(distance=150, available_time_start=1, available_time_end=8, battery_capacity=300, current_charge=100, charging_speed=15, efficiency=20),
    Vehicle(distance=80, available_time_start=2, available_time_end=10, battery_capacity=150, current_charge=30, charging_speed=8, efficiency=20)
]

hourly_prices = [8, 10, 12, 15, 12, 10, 8, 10, 12, 15, 10, 8, 10, 12, 15, 10, 8, 12, 15, 12, 10, 8, 10, 12]

charging_schedule, total_cost = optimize_charging_schedule(vehicles, hourly_prices)

# Print the charging schedule and total cost
print("Optimal Charging Schedule:")
for vehicle, schedule in charging_schedule:
    print("Vehicle:", vehicle.distance)
    print("Start Time:", vehicle.available_time_start)
    print("End Time:", vehicle.available_time_end)
    print("Charging Schedule:")
    for hour, energy in schedule:
        print("Hour:", hour, "| Energy Charged:", energy)
    print("----------------------")

print("Total Cost:", total_cost)


## Version 2: Multiple chargers are taken into account.
Managing to prevent overlapping of vehicles can be challenging.

In [ ]:
from pulp import LpVariable, LpBinary, LpProblem, LpMinimize, lpSum, value
import random

class Vehicle:
    def __init__(self, distance, available_time_start, available_time_end, charging_speed, efficiency):
        self.distance = distance
        self.available_time_start = available_time_start
        self.available_time_end = available_time_end
        self.charging_speed = charging_speed
        self.efficiency = efficiency

class Charger:
    def __init__(self, charger_id):
        self.charger_id = charger_id
        self.vehicles_assigned = []

    def is_available_at_hour(self, hour):
        return all(vehicle.available_time_start > hour or vehicle.available_time_end < hour or vehicle not in self.vehicles_assigned for vehicle in self.vehicles_assigned)

def generate_random_schedule(vehicle):
    # Generate a random charging schedule within the available time window
    schedule = {}
    for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
        var_name = f"Vehicle_{vehicle.distance}_{hour}"
        var = LpVariable(var_name, cat=LpBinary)
        schedule[hour] = var
    return schedule

def optimize_charging_schedule(vehicles, chargers, hourly_prices):
    problem = LpProblem("Charging_Schedule_Optimization", LpMinimize)

    charged_vars = LpVariable.dicts("Charged", ((vehicle, charger, hour) for vehicle in vehicles for charger in chargers for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)), cat=LpBinary)

    for vehicle in vehicles:
        schedule = generate_random_schedule(vehicle)

        # Restriction: The charged energy must be sufficient to cover the distance
        required_energy = vehicle.distance / (vehicle.efficiency / 100)
        problem += lpSum(charged_vars[(vehicle, charger, hour)] * vehicle.charging_speed for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1) for charger in chargers) >= required_energy

        # Ensure that each vehicle is assigned to a single charger at each hour
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            problem += lpSum(charged_vars[(vehicle, charger, hour)] for charger in chargers) == 1

        # Assign the vehicle to an available charger at each hour
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            chargers_available_at_hour = [charger for charger in chargers if charger.is_available_at_hour(hour)]
            if chargers_available_at_hour:
                assigned_charger = random.choice(chargers_available_at_hour)
                problem += charged_vars[(vehicle, assigned_charger, hour)] == 1
                assigned_charger.vehicles_assigned.append(vehicle)

    # Constraint: calculate total cost based on charging schedules and hourly prices
    total_cost = LpVariable("Total_Cost", lowBound=0)
    for vehicle in vehicles:
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            problem += lpSum(charged_vars[(vehicle, charger, hour)] * hourly_prices[hour] for charger in chargers) == total_cost

    # Solve the problem
    problem.solve()

    # Extract the charging schedules and total cost
    charging_schedule = []
    for charger in chargers:
        for vehicle in charger.vehicles_assigned:
            schedule = [(hour, charged_vars[(vehicle, charger, hour)].varValue) for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)]
            charging_schedule.append((vehicle, charger, schedule))

    total_cost = value(total_cost)

    return charging_schedule, total_cost

# Example usage
vehicles = [
    Vehicle(distance=100, available_time_start=0, available_time_end=6, charging_speed=10, efficiency=20),
    Vehicle(distance=150, available_time_start=1, available_time_end=8, charging_speed=15, efficiency=20),
    Vehicle(distance=80, available_time_start=2, available_time_end=10, charging_speed=8, efficiency=20)
]

chargers = [Charger(charger_id=1), Charger(charger_id=2), Charger(charger_id=3)]

hourly_prices = [8, 10, 12, 15, 12, 10, 8, 10, 12, 15, 10, 8, 10, 12, 15, 10, 8, 12, 15, 12, 10, 8, 10, 12]

charging_schedule, total_cost = optimize_charging_schedule(vehicles, chargers, hourly_prices)

# Print the charging schedule and total cost
print("Optimal Charging Schedule:")
for vehicle, charger, schedule in charging_schedule:
    print("Vehicle:", vehicle.distance)
    print("Start Time:", vehicle.available_time_start)
    print("End Time:", vehicle.available_time_end)
    print("Charger ID:", charger.charger_id)
    print("Charging Schedule:")
    for hour, charged in schedule:
        print("Hour:", hour, "| Charged:", charged)
    print("----------------------")

print("Total Cost:", total_cost)


In [ ]:
from pulp import LpVariable, LpBinary, LpProblem, LpMinimize, lpSum, value
import random

class Vehicle:
    def __init__(self, distance, available_time_start, available_time_end, charging_speed, efficiency):
        self.distance = distance
        self.available_time_start = available_time_start
        self.available_time_end = available_time_end
        self.charging_speed = charging_speed
        self.efficiency = efficiency

class Charger:
    def __init__(self, charger_id):
        self.charger_id = charger_id
        self.vehicles_assigned = []

def generate_random_schedule(vehicle):
    # Generación de la planificación aleatoria
    schedule = {}
    for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
        var_name = f"Vehicle_{vehicle.distance}_{hour}"
        var = LpVariable(var_name, cat=LpBinary)
        schedule[hour] = var
    return schedule

def optimize_charging_schedule(vehicles, chargers, hourly_prices):
    problem = LpProblem("Charging_Schedule_Optimization", LpMinimize)

    charged_vars = LpVariable.dicts("Charged", ((vehicle, charger, hour) for vehicle in vehicles for charger in chargers for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)), cat=LpBinary)

    for vehicle in vehicles:
        schedule = generate_random_schedule(vehicle)

        # Restriction: La carga debe ser suficiente para recorrer la distancia
        required_energy = vehicle.distance / (vehicle.efficiency / 100)
        problem += lpSum(charged_vars[(vehicle, charger, hour)] * vehicle.charging_speed for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1) for charger in chargers) >= required_energy

        # Asegurarse de que en cada cargador solo hay conectado un vehículo a la vez
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            problem += lpSum(charged_vars[(vehicle, charger, hour)] for charger in chargers) == 1

    # Constraint:Calcular el coste total basado en la carga y precios por hora
    total_cost = LpVariable("Total_Cost", lowBound=0)
    for vehicle in vehicles:
        for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1):
            problem += lpSum(charged_vars[(vehicle, charger, hour)] * hourly_prices[hour] for charger in chargers) == total_cost

    # Constraint: each charger can charge only one vehicle at a time in each hour
    for charger in chargers:
        for hour in range(max(vehicle.available_time_start for vehicle in vehicles), min(vehicle.available_time_end for vehicle in vehicles) + 1):
            problem += lpSum(charged_vars[(vehicle, charger, hour)] for vehicle in vehicles) <= 1

    # Solve the problem
    problem.solve()

    # Show schedule
    charging_schedule = []
    for charger in chargers:
        for vehicle in vehicles:
            if any(charged_vars[(vehicle, charger, hour)].varValue == 1 for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)):
                schedule = [(hour, charged_vars[(vehicle, charger, hour)].varValue) for hour in range(vehicle.available_time_start, vehicle.available_time_end + 1)]
                charging_schedule.append((vehicle, charger, schedule))

    total_cost = value(total_cost)

    return charging_schedule, total_cost

# Example usage
vehicles = [
    Vehicle(distance=100, available_time_start=0, available_time_end=6, charging_speed=10, efficiency=20),
    Vehicle(distance=150, available_time_start=1, available_time_end=8, charging_speed=15, efficiency=20),
    Vehicle(distance=80, available_time_start=2, available_time_end=10, charging_speed=8, efficiency=20)
]

chargers = [Charger(charger_id=1), Charger(charger_id=2), Charger(charger_id=3)]

hourly_prices = [8, 10, 12, 15, 12, 10, 8, 10, 12, 15, 10, 8, 10, 12, 15, 10, 8, 12, 15, 12, 10, 8, 10, 12]

charging_schedule, total_cost = optimize_charging_schedule(vehicles, chargers, hourly_prices)

# Print the charging schedule and total cost
print("Optimal Charging Schedule:")
for vehicle, charger, schedule in charging_schedule:
    print("Vehicle:", vehicle.distance)
    print("Start Time:", vehicle.available_time_start)
    print("End Time:", vehicle.available_time_end)
    print("Charger ID:", charger.charger_id)
    print("Charging Schedule:")
    for hour, charged in schedule:
        print("Hour:", hour, "| Charged:", charged)
    print("----------------------")

print("Total Cost:", total_cost)


## Version 3: Change in the implementation of the linear optimization algorithm with constraints (this is the version used in experiments)

Similar approach to the genetic algorithm, changes the way of addressing the problem and the format of the solution. Takes into account multiple chargers. The first algorithm does not calculate the exact time required for vehicle charging, while the second does. Both algorithms do not consider uncertainty.

In [ ]:

import random
from pulp import *

# Restricciones lineales para asegurar que los cargadores no se solapan
def check_cargador_constraints(assignment):
    for i in range(len(assignment)):
        for j in range(i + 1, len(assignment)):
            if assignment[i].cargador == assignment[j].cargador:
                if assignment[i].tiempo_fin > assignment[j].tiempo_inicio and assignment[j].tiempo_fin > assignment[i].tiempo_inicio:
                    return False
    return True

# Función para resolver el problema mediante programación lineal
def solve_optimization_problem(vehiculos, cargadores, hourly_prices):
    # Crear un problema de programación lineal
    prob = LpProblem("Carga_de_vehiculos", LpMinimize)

    # Variables de decisión
    assignment_vars = LpVariable.dicts("Assignment", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, 1, LpBinary)

    # Función objetivo (minimizar el coste total de carga)
    prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] * vehiculo.get_charging_cost(vehiculo.available_time_start, vehiculo.available_time_end, hourly_prices) for vehiculo in vehiculos for cargador in cargadores])

    # Restricciones: cada vehículo es asignado a exactamente un cargador
    for vehiculo in vehiculos:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for cargador in cargadores]) == 1

    # Restricciones: cada cargador solo puede cargar un vehículo a la vez
    for cargador in cargadores:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for vehiculo in vehiculos]) <= 1

    # Restricciones adicionales para asegurar que los cargadores no se solapen
    for vehiculo1 in vehiculos:
        for vehiculo2 in vehiculos:
            if vehiculo1 != vehiculo2:
                for cargador in cargadores:
                    prob += assignment_vars[(vehiculo1.id, cargador.id)] + assignment_vars[(vehiculo2.id, cargador.id)] <= 1

    # Resolver el problema de programación lineal
    prob.solve()

    # Extraer la solución
    best_assignment = []
    for vehiculo in vehiculos:
        for cargador in cargadores:
            if assignment_vars[(vehiculo.id, cargador.id)].value() == 1:
                best_assignment.append(item_planlist(vehiculo, vehiculo.available_time_start * assignment_vars[(vehiculo.id, cargador.id)].value(), vehiculo.available_time_end * assignment_vars[(vehiculo.id, cargador.id)].value(), cargador))

    # Calcular el coste total de la solución encontrada
    best_cost = value(prob.objective)

    return best_assignment, best_cost

# Parámetros del algoritmo
hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

# Ejemplo de vehículos
vehiculo1 = ElectricVehicle(1, 100, 8, 9, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculos = [vehiculo1, vehiculo2, vehiculo3]

# Ejemplo de cargadores
cargadores = [Cargador(1), Cargador(2), Cargador(3)]


# Resolver el problema mediante programación lineal
best_assignment, best_cost = solve_optimization_problem(vehiculos, cargadores, hourly_prices)



# Imprimir la mejor solución encontrada
print("Coste total: " + str(best_cost))
print("Planificación:")
for item in best_assignment:
    print("Vehículo " + str(item.vehiculo.id))
    print("Inicio: " + str(item.tiempo_inicio))
    print("Fin: " + str(item.tiempo_fin))
    print("Cargador " + str(item.cargador.id))
    print("")



Same algorithm but using data file

In [ ]:
import random
from pulp import *

# Linear constraints to ensure that chargers do not overlap
def check_cargador_constraints(assignment):
    for i in range(len(assignment)):
        for j in range(i + 1, len(assignment)):
            if assignment[i].cargador == assignment[j].cargador:
                if assignment[i].tiempo_fin > assignment[j].tiempo_inicio and assignment[j].tiempo_fin > assignment[i].tiempo_inicio:
                    return False
    return True

# Function to obtain the exact charging time for a vehicle
def get_exact_charging_time(vehiculo, begin_time, end_time):
    tiempo_inicio = begin_time
    tiempo_fin = end_time

    required_charge = vehiculo.distance * (vehiculo.discharge_rate / 100)  # Cálculo de la carga requerida para la distancia a recorrer

    # f the car does not have sufficient charge
    if vehiculo.current_charge < required_charge:
        required_charge -= vehiculo.current_charge  # Deduct the current charge
        tiempo_fin_carga = tiempo_inicio + required_charge / vehiculo.charge_speed  # Time in hours required to charge the vehicle.

        if tiempo_fin_carga > tiempo_fin:
            tiempo_fin_carga = tiempo_fin
    else:
        tiempo_fin_carga = tiempo_inicio

    return tiempo_fin_carga

# Function to solve the problem using linear programming
def solve_optimization_problem(vehiculos, cargadores, hourly_prices):
    # Create a linear programming problem
    prob = LpProblem("Carga_de_vehiculos", LpMinimize)

    # Variables de decisión
    assignment_vars = LpVariable.dicts("Assignment", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, 1, LpBinary)

    # Decision variables
    prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] * vehiculo.get_charging_cost(vehiculo.available_time_start, vehiculo.available_time_end, hourly_prices) for vehiculo in vehiculos for cargador in cargadores])

    # Constraints: Each vehicle is assigned to exactly one charger
    for vehiculo in vehiculos:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for cargador in cargadores]) == 1

    # Constraints: Each charger can only charge one vehicle at a time
    for cargador in cargadores:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for vehiculo in vehiculos]) <= 1

    # Additional constraints to ensure that chargers do not overlap
    for vehiculo1 in vehiculos:
        for vehiculo2 in vehiculos:
            if vehiculo1 != vehiculo2:
                for cargador in cargadores:
                    prob += assignment_vars[(vehiculo1.id, cargador.id)] + assignment_vars[(vehiculo2.id, cargador.id)] <= 1

    # Solve the problem
    prob.solve()

    # Extract solution
    best_assignment = []
    for vehiculo in vehiculos:
        for cargador in cargadores:
            if assignment_vars[(vehiculo.id, cargador.id)].value() == 1:
                tiempo_inicio = vehiculo.available_time_start * assignment_vars[(vehiculo.id, cargador.id)].value()
                tiempo_fin = vehiculo.available_time_end * assignment_vars[(vehiculo.id, cargador.id)].value()
                tiempo_fin_carga = get_exact_charging_time(vehiculo, tiempo_inicio, tiempo_fin)
                best_assignment.append(item_planlist(vehiculo, tiempo_inicio, tiempo_fin_carga, cargador))

    # Calculate total cost
    best_cost = value(prob.objective)
    lista_ordenada = sorted(best_assignment, key=lambda x: x.vehiculo.id)

    return lista_ordenada, best_cost

# parameters
hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

# vehicles example - uncoment to add more vehicles
'''
vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 13, 1, 120, 12, 6)
vehiculo5 = ElectricVehicle(5, 100, 14, 17, 2, 80, 10, 5)
vehiculo6 = ElectricVehicle(6, 200, 13, 18, 3, 100, 8, 4)
vehiculo7 = ElectricVehicle(7, 150, 12, 20, 1, 120, 12, 6)
vehiculo8 = ElectricVehicle(8, 200, 15, 22, 1, 120, 12, 6)
vehiculo9 = ElectricVehicle(9, 100, 9, 10, 2, 80, 10, 5)
vehiculo10 = ElectricVehicle(10, 200, 8, 20, 3, 100, 8, 4)
vehiculo11 = ElectricVehicle(11, 150, 12, 18, 1, 120, 12, 6)
vehiculo12 = ElectricVehicle(12, 200, 14, 20, 1, 120, 12, 6)
vehiculo13 = ElectricVehicle(13, 100, 21, 22, 2, 80, 10, 5)
vehiculo14 = ElectricVehicle(14, 200, 22, 23, 3, 100, 8, 4)
vehiculo15 = ElectricVehicle(15, 150, 20, 23, 1, 120, 12, 6)
vehiculo16 = ElectricVehicle(16, 200, 15, 22, 1, 120, 12, 6)
vehiculo17 = ElectricVehicle(17, 100, 9, 20, 2, 80, 10, 5)
vehiculo18 = ElectricVehicle(18, 200, 6, 10, 3, 100, 8, 4)
vehiculo19 = ElectricVehicle(19, 150, 4, 9, 1, 120, 12, 6)
vehiculo20 = ElectricVehicle(20, 200, 17, 23, 1, 120, 12, 6)
vehiculo21 = ElectricVehicle(21, 100, 14, 18, 2, 80, 10, 5)
vehiculo22 = ElectricVehicle(22, 200, 18, 23, 3, 100, 8, 4)
vehiculo23 = ElectricVehicle(23, 150, 12, 15, 1, 120, 12, 6)
vehiculo24 = ElectricVehicle(24, 200, 15, 22, 1, 120, 12, 6)
vehiculo25 = ElectricVehicle(25, 100, 2, 10, 2, 80, 10, 5)
vehiculo26 = ElectricVehicle(26, 200, 3, 12, 3, 100, 8, 4)
vehiculo27 = ElectricVehicle(27, 150, 6, 7, 1, 120, 12, 6)
vehiculo28 = ElectricVehicle(28, 200, 7, 18, 1, 120, 12, 6)
vehiculo29 = ElectricVehicle(29, 100, 21, 22, 2, 80, 10, 5)
vehiculo30 = ElectricVehicle(30, 200, 5, 7, 3, 100, 8, 4)
vehiculo31 = ElectricVehicle(31, 150, 6, 8, 1, 120, 12, 6)
vehiculo32 = ElectricVehicle(32, 200, 5, 18, 1, 120, 12, 6)
vehiculo33 = ElectricVehicle(33, 100, 9, 10, 2, 80, 10, 5)
vehiculo34 = ElectricVehicle(34, 200, 8, 9, 3, 100, 8, 4)
vehiculo35 = ElectricVehicle(35, 150, 12, 13, 1, 120, 12, 6)
vehiculo36 = ElectricVehicle(36, 200, 12, 13, 1, 120, 12, 6)
vehiculo37 = ElectricVehicle(37, 100, 14, 17, 2, 80, 10, 5)
vehiculo38 = ElectricVehicle(38, 200, 13, 18, 3, 100, 8, 4)
vehiculo39 = ElectricVehicle(39, 150, 12, 20, 1, 120, 12, 6)
vehiculo40 = ElectricVehicle(40, 200, 15, 22, 1, 120, 12, 6)
vehiculo41 = ElectricVehicle(41, 100, 9, 10, 2, 80, 10, 5)
vehiculo42 = ElectricVehicle(42, 200, 8, 20, 3, 100, 8, 4)
vehiculo43 = ElectricVehicle(43, 150, 12, 18, 1, 120, 12, 6)
vehiculo44 = ElectricVehicle(44, 200, 14, 20, 1, 120, 12, 6)
vehiculo45 = ElectricVehicle(45, 100, 21, 22, 2, 80, 10, 5)
vehiculo46 = ElectricVehicle(46, 200, 22, 23, 3, 100, 8, 4)
vehiculo47 = ElectricVehicle(47, 150, 20, 23, 1, 120, 12, 6)
vehiculo48 = ElectricVehicle(48, 200, 15, 22, 1, 120, 12, 6)
vehiculo49 = ElectricVehicle(49, 100, 9, 20, 2, 80, 10, 5)
vehiculo50 = ElectricVehicle(50, 200, 6, 10, 3, 100, 8, 4)
vehiculo51 = ElectricVehicle(51, 150, 4, 9, 1, 120, 12, 6)
vehiculo52 = ElectricVehicle(52, 200, 17, 23, 1, 120, 12, 6)
vehiculo53 = ElectricVehicle(53, 100, 14, 18, 2, 80, 10, 5)
vehiculo54 = ElectricVehicle(54, 200, 18, 23, 3, 100, 8, 4)
vehiculo55 = ElectricVehicle(55, 150, 12, 15, 1, 120, 12, 6)
vehiculo56 = ElectricVehicle(56, 200, 15, 22, 1, 120, 12, 6)
vehiculo57 = ElectricVehicle(57, 100, 2, 10, 2, 80, 10, 5)
vehiculo58 = ElectricVehicle(58, 200, 3, 12, 3, 100, 8, 4)
vehiculo59 = ElectricVehicle(59, 150, 6, 7, 1, 120, 12, 6)
vehiculo60 = ElectricVehicle(60, 200, 7, 18, 1, 120, 12, 6)
vehiculo61 = ElectricVehicle(61, 100, 21, 22, 2, 80, 10, 5)
vehiculo62 = ElectricVehicle(62, 200, 5, 7, 3, 100, 8, 4)
vehiculo63 = ElectricVehicle(63, 150, 6, 8, 1, 120, 12, 6)
vehiculo64 = ElectricVehicle(64, 200, 5, 18, 1, 120, 12, 6)
vehiculo65 = ElectricVehicle(65, 100, 9, 10, 2, 80, 10, 5)
vehiculo66 = ElectricVehicle(66, 200, 8, 9, 3, 100, 8, 4)
vehiculo67 = ElectricVehicle(67, 150, 12, 13, 1, 120, 12, 6)
vehiculo68 = ElectricVehicle(68, 200, 12, 13, 1, 120, 12, 6)
vehiculo69 = ElectricVehicle(69, 100, 14, 17, 2, 80, 10, 5)
vehiculo70 = ElectricVehicle(70, 200, 13, 18, 3, 100, 8, 4)
vehiculo71 = ElectricVehicle(71, 150, 12, 20, 1, 120, 12, 6)
vehiculo72 = ElectricVehicle(72, 200, 15, 22, 1, 120, 12, 6)
vehiculo73 = ElectricVehicle(73, 100, 9, 10, 2, 80, 10, 5)
vehiculo74 = ElectricVehicle(74, 200, 8, 20, 3, 100, 8, 4)
vehiculo75 = ElectricVehicle(75, 150, 12, 18, 1, 120, 12, 6)
vehiculo76 = ElectricVehicle(76, 200, 14, 20, 1, 120, 12, 6)
vehiculo77 = ElectricVehicle(77, 100, 21, 22, 2, 80, 10, 5)
vehiculo78 = ElectricVehicle(78, 200, 22, 23, 3, 100, 8, 4)
vehiculo79 = ElectricVehicle(79, 150, 20, 23, 1, 120, 12, 6)
vehiculo80 = ElectricVehicle(80, 200, 15, 22, 1, 120, 12, 6)
vehiculo81 = ElectricVehicle(81, 100, 9, 20, 2, 80, 10, 5)
vehiculo82 = ElectricVehicle(82, 200, 6, 10, 3, 100, 8, 4)
vehiculo83 = ElectricVehicle(83, 150, 4, 9, 1, 120, 12, 6)
vehiculo84 = ElectricVehicle(84, 200, 17, 23, 1, 120, 12, 6)
vehiculo85 = ElectricVehicle(85, 100, 14, 18, 2, 80, 10, 5)
vehiculo86 = ElectricVehicle(86, 200, 18, 23, 3, 100, 8, 4)
vehiculo87 = ElectricVehicle(87, 150, 12, 15, 1, 120, 12, 6)
vehiculo88 = ElectricVehicle(88, 200, 15, 22, 1, 120, 12, 6)
vehiculo89 = ElectricVehicle(89, 100, 2, 10, 2, 80, 10, 5)
vehiculo90 = ElectricVehicle(90, 200, 3, 12, 3, 100, 8, 4)
vehiculo91 = ElectricVehicle(91, 150, 6, 7, 1, 120, 12, 6)
vehiculo92 = ElectricVehicle(92, 200, 7, 18, 1, 120, 12, 6)
vehiculo93 = ElectricVehicle(93, 100, 21, 22, 2, 80, 10, 5)
vehiculo94 = ElectricVehicle(94, 200, 5, 7, 3, 100, 8, 4)
vehiculo95 = ElectricVehicle(95, 150, 6, 8, 1, 120, 12, 6)
vehiculo96 = ElectricVehicle(96, 200, 5, 18, 1, 120, 12, 6)
vehiculo97 = ElectricVehicle(97, 100, 9, 10, 2, 80, 10, 5)
vehiculo98 = ElectricVehicle(98, 200, 8, 9, 3, 100, 8, 4)
vehiculo99 = ElectricVehicle(99, 150, 12, 13, 1, 120, 12, 6)
vehiculo100 = ElectricVehicle(100, 200, 12, 13, 1, 120, 12, 6)
vehiculo101 = ElectricVehicle(101, 100, 14, 17, 2, 80, 10, 5)
vehiculo102 = ElectricVehicle(102, 200, 13, 18, 3, 100, 8, 4)
vehiculo103 = ElectricVehicle(103, 150, 12, 20, 1, 120, 12, 6)
vehiculo104 = ElectricVehicle(104, 200, 15, 22, 1, 120, 12, 6)
vehiculo105 = ElectricVehicle(105, 100, 9, 10, 2, 80, 10, 5)
vehiculo106 = ElectricVehicle(106, 200, 8, 20, 3, 100, 8, 4)
vehiculo107 = ElectricVehicle(107, 150, 12, 18, 1, 120, 12, 6)
vehiculo108 = ElectricVehicle(108, 200, 14, 20, 1, 120, 12, 6)
vehiculo109 = ElectricVehicle(109, 100, 21, 22, 2, 80, 10, 5)
vehiculo110 = ElectricVehicle(110, 200, 22, 23, 3, 100, 8, 4)
vehiculo111 = ElectricVehicle(111, 150, 20, 23, 1, 120, 12, 6)
vehiculo112 = ElectricVehicle(112, 200, 15, 22, 1, 120, 12, 6)
vehiculo113 = ElectricVehicle(113, 100, 9, 20, 2, 80, 10, 5)
vehiculo114 = ElectricVehicle(114, 200, 6, 10, 3, 100, 8, 4)
vehiculo115 = ElectricVehicle(115, 150, 4, 9, 1, 120, 12, 6)
vehiculo116 = ElectricVehicle(116, 200, 17, 23, 1, 120, 12, 6)
vehiculo117 = ElectricVehicle(117, 100, 14, 18, 2, 80, 10, 5)
vehiculo118 = ElectricVehicle(118, 200, 18, 23, 3, 100, 8, 4)
vehiculo119 = ElectricVehicle(119, 150, 12, 15, 1, 120, 12, 6)
vehiculo120 = ElectricVehicle(120, 200, 15, 22, 1, 120, 12, 6)
vehiculo121 = ElectricVehicle(121, 100, 9, 10, 2, 80, 10, 5)
vehiculo122 = ElectricVehicle(122, 200, 8, 9, 3, 100, 8, 4)
vehiculo123 = ElectricVehicle(123, 150, 12, 13, 1, 120, 12, 6)
vehiculo124 = ElectricVehicle(124, 200, 12, 13, 1, 120, 12, 6)
vehiculo125 = ElectricVehicle(125, 100, 14, 17, 2, 80, 10, 5)
vehiculo126 = ElectricVehicle(126, 200, 13, 18, 3, 100, 8, 4)
vehiculo127 = ElectricVehicle(127, 150, 12, 20, 1, 120, 12, 6)
vehiculo128 = ElectricVehicle(128, 200, 15, 22, 1, 120, 12, 6)
vehiculo129 = ElectricVehicle(129, 100, 9, 10, 2, 80, 10, 5)
vehiculo130 = ElectricVehicle(130, 200, 8, 20, 3, 100, 8, 4)
vehiculo131 = ElectricVehicle(131, 150, 12, 18, 1, 120, 12, 6)
vehiculo132 = ElectricVehicle(132, 200, 14, 20, 1, 120, 12, 6)
vehiculo133 = ElectricVehicle(133, 100, 21, 22, 2, 80, 10, 5)
vehiculo134 = ElectricVehicle(134, 200, 22, 23, 3, 100, 8, 4)
vehiculo135 = ElectricVehicle(135, 150, 20, 23, 1, 120, 12, 6)
vehiculo136 = ElectricVehicle(136, 200, 15, 22, 1, 120, 12, 6)
vehiculo137 = ElectricVehicle(137, 100, 9, 20, 2, 80, 10, 5)
vehiculo138 = ElectricVehicle(138, 200, 6, 10, 3, 100, 8, 4)
vehiculo139 = ElectricVehicle(139, 150, 4, 9, 1, 120, 12, 6)
vehiculo140 = ElectricVehicle(140, 200, 17, 23, 1, 120, 12, 6)
vehiculo141 = ElectricVehicle(141, 100, 14, 18, 2, 80, 10, 5)
vehiculo142 = ElectricVehicle(142, 200, 18, 23, 3, 100, 8, 4)
vehiculo143 = ElectricVehicle(143, 150, 12, 15, 1, 120, 12, 6)
vehiculo144 = ElectricVehicle(144, 200, 15, 22, 1, 120, 12, 6)
vehiculo145 = ElectricVehicle(145, 100, 2, 10, 2, 80, 10, 5)
vehiculo146 = ElectricVehicle(146, 200, 3, 12, 3, 100, 8, 4)
vehiculo147 = ElectricVehicle(147, 150, 6, 7, 1, 120, 12, 6)
vehiculo148 = ElectricVehicle(148, 200, 7, 18, 1, 120, 12, 6)
vehiculo149 = ElectricVehicle(149, 100, 21, 22, 2, 80, 10, 5)
vehiculo150 = ElectricVehicle(150, 200, 5, 7, 3, 100, 8, 4)'''



#vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4, vehiculo5, vehiculo6, vehiculo7, vehiculo8, vehiculo9, vehiculo10]
def create_vehicles_from_file(file_name):
    vehicles = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.strip().split(', ')
            if len(data) == 8:
                id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate = map(float, data)
                vehicle = ElectricVehicle(id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate)
                vehicles.append(vehicle)
    return vehicles

file_name = 'vehiculos.txt'
vehiculos = create_vehicles_from_file(file_name)



# Chargers
cargadores = [Cargador(1), Cargador(2), Cargador(3),Cargador(1), Cargador(2), Cargador(3)]

import time
inicial = time.time()

# Solve the problem
best_assignment, best_cost = solve_optimization_problem(vehiculos, cargadores, hourly_prices)

final = time.time()

# Calculate timespan
tiempo_transcurrido = final - inicial

# Print solution
print("Coste total: " + str(best_cost/3))
print("Planificación:")
for item in best_assignment:
    print("Vehículo " + str(item.vehiculo.id))
    print("Inicio: " + str(item.tiempo_inicio))
    print("Fin: " + str(item.tiempo_fin))
    print("Cargador " + str(item.cargador.id))
    print("")


print(f"Tiempo de ejecución: {tiempo_transcurrido * 1000:.2f} milisegundos")


#STATISTICS

'''file_name = 'vehiculosprueba.txt'
vehiculos2 = create_vehicles_from_file(file_name)


i=0
cont=0
vehiculos_noc=0
tiempo_real_carga=0
coste_real_total = 0
coste_real=0
coste_penalizacion=0
timespan_penalizacion=0
for item in best_assignment:

  if item.vehiculo.id == vehiculos2[i].id:

    if item.tiempo_inicio<vehiculos2[i].available_time_start:
      cont=cont+1
      tiempo_real_carga = item.tiempo_fin - vehiculos2[i].available_time_start
      tiempo_real_ini= vehiculos2[i].available_time_start

      if vehiculos[i].get_charging_time()>tiempo_real_carga:
          vehiculos_noc=vehiculos_noc+1
          tiempo_penalizacion=vehiculos[i].get_charging_time()-tiempo_real_carga
          #Para los vehiculos que por llegar tarde no se carguen lo suficiente, se añade una penalización sumando el coste de cargar lo que les falta en el momento más caro del día
          #Tiempo que queda x velocidad de carga x precio de hora más cara
          coste_penalizacion=coste_penalizacion+(tiempo_penalizacion*vehiculos[i].charge_speed)*0.35

    else:
      tiempo_real_carga=item.tiempo_fin-item.tiempo_inicio
      tiempo_real_ini=item.tiempo_inicio

    coste_real=vehiculos[i].get_charging_cost(item.tiempo_inicio, item.tiempo_fin, hourly_prices)


    timespan_inicio=0
    timespan_fin=0

    if i==0:
      timespan_inicio=tiempo_real_ini

    else: timespan_fin=item.tiempo_fin+tiempo_penalizacion

  i=i+1
  coste_real_total = coste_real_total + coste_real/2 ++coste_penalizacion
  #timespan=timespan_fin-timespan_inicio

print("Número de intentos de carga y que no esté el vehículo")
print(cont)

print("Número de vehículos que no se cargan lo suficiente")
print(vehiculos_noc)

print("Coste real total")
print(coste_real_total)

print("Timespan real")
#print(timespan)'''


In [ ]:
import random
from pulp import *

# Restricciones lineales para asegurar que los cargadores no se solapan
def check_cargador_constraints(assignment):
    for i in range(len(assignment)):
        for j in range(i + 1, len(assignment)):
            if assignment[i].cargador == assignment[j].cargador:
                if assignment[i].tiempo_fin > assignment[j].tiempo_inicio and assignment[j].tiempo_fin > assignment[i].tiempo_inicio:
                    return False
    return True

# Función para obtener el tiempo exacto de carga para un vehículo
def get_exact_charging_time(vehiculo, begin_time, end_time):
    tiempo_inicio = begin_time
    tiempo_fin = end_time

    required_charge = vehiculo.distance * (vehiculo.discharge_rate / 100)  # Cálculo de la carga requerida para la distancia a recorrer

    # Si el coche no tiene carga suficiente
    if vehiculo.current_charge < required_charge:
        required_charge -= vehiculo.current_charge  # Restar la carga actual
        tiempo_fin_carga = tiempo_inicio + required_charge / vehiculo.charge_speed  # Tiempo en horas requerido para cargar el vehículo

        if tiempo_fin_carga > tiempo_fin:
            tiempo_fin_carga = tiempo_fin
    else:
        tiempo_fin_carga = tiempo_inicio

    return tiempo_fin_carga



# Función para resolver el problema mediante programación lineal con ponderación
def solve_optimization_problem_with_weight(vehiculos, cargadores, hourly_prices, weight_cost, weight_timespan):
    # Crear un problema de programación lineal
    prob = LpProblem("Carga_de_vehiculos", LpMinimize)

    # Variables de decisión
    assignment_vars = LpVariable.dicts("Assignment", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, 1, LpBinary)
    start_vars = LpVariable.dicts("Start", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, None, LpContinuous)
    end_vars = LpVariable.dicts("End", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, None, LpContinuous)
    delta_vars = LpVariable.dicts("Delta", [(vehiculo.id, cargador.id) for vehiculo in vehiculos for cargador in cargadores], 0, None, LpContinuous)

    # Función objetivo ponderada (minimizar el coste total y el tiempo total)
    prob += (weight_cost * lpSum([assignment_vars[(vehiculo.id, cargador.id)] * vehiculo.get_charging_cost(vehiculo.available_time_start, vehiculo.available_time_end, hourly_prices) for vehiculo in vehiculos for cargador in cargadores])
             + weight_timespan * lpSum([delta_vars[(vehiculo.id, cargador.id)] for vehiculo in vehiculos for cargador in cargadores]))

    # Restricciones: cada vehículo es asignado a exactamente un cargador
    for vehiculo in vehiculos:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for cargador in cargadores]) == 1

    # Restricciones: cada cargador solo puede cargar un vehículo a la vez
    for cargador in cargadores:
        prob += lpSum([assignment_vars[(vehiculo.id, cargador.id)] for vehiculo in vehiculos]) <= 1

    # Restricciones adicionales para asegurar que los cargadores no se solapen
    for vehiculo1 in vehiculos:
        for vehiculo2 in vehiculos:
            if vehiculo1 != vehiculo2:
                for cargador in cargadores:
                    prob += assignment_vars[(vehiculo1.id, cargador.id)] + assignment_vars[(vehiculo2.id, cargador.id)] <= 1

    # Restricciones para el tiempo exacto de inicio y fin de carga
    for vehiculo in vehiculos:
        for cargador in cargadores:
            prob += start_vars[(vehiculo.id, cargador.id)] >= vehiculo.available_time_start * assignment_vars[(vehiculo.id, cargador.id)]
            prob += end_vars[(vehiculo.id, cargador.id)] <= vehiculo.available_time_end * assignment_vars[(vehiculo.id, cargador.id)]
            prob += delta_vars[(vehiculo.id, cargador.id)] >= end_vars[(vehiculo.id, cargador.id)] - start_vars[(vehiculo.id, cargador.id)]
            prob += delta_vars[(vehiculo.id, cargador.id)] >= 0
            prob += delta_vars[(vehiculo.id, cargador.id)] <= vehiculo.distance / vehiculo.charge_speed * assignment_vars[(vehiculo.id, cargador.id)]

    # Resolver el problema de programación lineal
    prob.solve()

    # Extraer la solución
    best_assignment = []
    for vehiculo in vehiculos:
        for cargador in cargadores:
            if assignment_vars[(vehiculo.id, cargador.id)].value() == 1:
                tiempo_inicio = start_vars[(vehiculo.id, cargador.id)].value()
                tiempo_fin = end_vars[(vehiculo.id, cargador.id)].value()
                best_assignment.append(item_planlist(vehiculo, tiempo_inicio, tiempo_fin, cargador))

    # Calcular el coste total y el tiempo total de la solución encontrada
    best_cost = value(prob.objective)
    total_timespan = sum([delta_vars[(vehiculo.id, cargador.id)].value() for vehiculo in vehiculos for cargador in cargadores])
    lista_ordenada = sorted(best_assignment, key=lambda x: x.vehiculo.id)

    return lista_ordenada, best_cost, total_timespan





# Parámetros del algoritmo
hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 13, 1, 120, 12, 6)
vehiculo5 = ElectricVehicle(5, 100, 14, 17, 2, 80, 10, 5)
vehiculo6 = ElectricVehicle(6, 200, 13, 18, 3, 100, 8, 4)
vehiculo7 = ElectricVehicle(7, 150, 12, 20, 1, 120, 12, 6)
vehiculo8 = ElectricVehicle(8, 200, 15, 22, 1, 120, 12, 6)
vehiculo9 = ElectricVehicle(9, 100, 9, 10, 2, 80, 10, 5)
vehiculo10 = ElectricVehicle(10, 200, 8, 20, 3, 100, 8, 4)

vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4, vehiculo5, vehiculo6, vehiculo7, vehiculo8, vehiculo9, vehiculo10]

# Ejemplo de cargadores
cargadores = [Cargador(1), Cargador(2), Cargador(3)]

# Parámetros de ponderación
weight_cost = 1.0  # Ponderación para el costo total
weight_timespan = 1.0  # Ponderación para el tiempo total

# Resolver el problema mediante programación lineal con ponderación
best_assignment, best_cost, total_timespan = solve_optimization_problem_with_weight(vehiculos, cargadores, hourly_prices, weight_cost, weight_timespan)

# Imprimir la mejor solución encontrada
print("Coste total: " + str(best_cost))
print("Tiempo total: " + str(total_timespan))
print("Planificación:")
for item in best_assignment:
    print("Vehículo " + str(item.vehiculo.id))
    print("Inicio: " + str(item.tiempo_inicio))
    print("Fin: " + str(item.tiempo_fin))
    print("Cargador " + str(item.cargador.id))
    print("")



## Chart visualization.

In [ ]:
import matplotlib.pyplot as plt

def generate_gantt_diagram(assignment, total_cost):
    fig, ax = plt.subplots()

    cargador_ids = list(set(item.cargador.id for item in assignment))
    vehiculo_ids = list(set(item.vehiculo.id for item in assignment))

    min_start_time = float('inf')
    max_end_time = float('-inf')

    for cargador_id in cargador_ids:
        for vehiculo_id in vehiculo_ids:
            tasks = [(item.tiempo_inicio, item.tiempo_fin - item.tiempo_inicio, item.vehiculo.id) for item in assignment if item.vehiculo.id == vehiculo_id and item.cargador.id == cargador_id]
            if tasks:
                tiempo_inicio, duracion, vehiculo_id = zip(*tasks)
                ax.broken_barh(list(zip(tiempo_inicio, duracion)), (cargador_id - 0.4, 0.8), facecolors='tab:blue', edgecolor='white')
                for i in range(len(tasks)):
                    ax.text(tiempo_inicio[i] + duracion[i] / 2, cargador_id, f'V{vehiculo_id[i]}', ha='center', va='center')

                min_start_time = min(min_start_time, min(tiempo_inicio))
                max_end_time = max(max_end_time, max(tiempo_inicio) + max(duracion))

    ax.set_xlabel('Tiempo')
    ax.set_ylabel('Cargadores')
    ax.set_yticks(cargador_ids)
    ax.set_yticklabels(['Cargador {}'.format(cargador_id) for cargador_id in cargador_ids])
    ax.set_title('Diagrama de Gantt para la asignación de vehículos a cargadores')

    # Agregar el coste total y el timespan en la parte superior del diagrama
    ax.text(0.5, 1.1, f'Coste Total: {total_cost}', transform=ax.transAxes, ha='center')
    ax.text(0.5, 1.05, f'Timespan: {max_end_time - min_start_time} horas', transform=ax.transAxes, ha='center')

    plt.tight_layout()
    plt.show()

# Ejemplo de uso con `best_assignment` y `best_cost` obtenidos del problema de programación lineal
generate_gantt_diagram(best_assignment, best_cost)


In [ ]:
import matplotlib.pyplot as plt

def generate_gantt_diagram(assignment, total_cost):
    fig, ax = plt.subplots()

    cargador_ids = list(set(item.cargador.id for item in assignment))
    vehiculo_ids = list(set(item.vehiculo.id for item in assignment))

    min_start_time = float('inf')
    max_end_time = float('-inf')

    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray']

    for cargador_id in cargador_ids:
        for vehiculo_id in vehiculo_ids:
            tasks = [(item.tiempo_inicio, item.tiempo_fin - item.tiempo_inicio, item.vehiculo.id) for item in assignment if item.vehiculo.id == vehiculo_id and item.cargador.id == cargador_id]
            if tasks:
                tiempo_inicio, duracion, vehiculo_id = zip(*tasks)
                ax.broken_barh(list(zip(tiempo_inicio, duracion)), (cargador_id - 0.4, 0.8), facecolors=colors[vehiculo_id[0] % len(colors)], edgecolor='white')
                for i in range(len(tasks)):
                    ax.text(tiempo_inicio[i] + duracion[i] / 2, cargador_id, f'V{vehiculo_id[i]}', ha='center', va='center')

                min_start_time = min(min_start_time, min(tiempo_inicio))
                max_end_time = max(max_end_time, max(tiempo_inicio) + max(duracion))

    ax.set_xlabel('Tiempo')
    ax.set_ylabel('Cargadores')
    ax.set_yticks(cargador_ids)
    ax.set_yticklabels(['Cargador {}'.format(cargador_id) for cargador_id in cargador_ids])
    ax.set_title('Diagrama de Gantt para la asignación de vehículos a cargadores')

    # Agregar el coste total y el timespan en la parte superior del diagrama
    ax.text(0.5, 1.15, f'Coste Total: {total_cost:.2f}', transform=ax.transAxes, ha='center')
    ax.text(0.5, 1.10, f'Timespan: {max_end_time - min_start_time:.2f} horas', transform=ax.transAxes, ha='center')

    plt.tight_layout()
    plt.show()

# Ejemplo de uso con `best_assignment` y `best_cost` obtenidos del problema de programación lineal
generate_gantt_diagram(best_assignment, best_cost)

